#Test load model dengan dataset Test

## Install dependencies

In [ ]:
pip install --no-deps tensorflowjs split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## import some of the dependencies which is being used

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array, ImageDataGenerator, load_img
from tensorflow.keras.layers import Dense, Input, Dropout, GlobalAveragePooling2D, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.optimizers import Adam, SGD, Adadelta
from tensorflow.keras.regularizers import l2
from tensorflow.keras.constraints import max_norm, unit_norm
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback, LearningRateScheduler,TensorBoard
from matplotlib.pyplot import imread, imshow, subplots, show
from sklearn.model_selection import train_test_split,KFold, StratifiedKFold,cross_val_score
from google.colab.patches import cv2_imshow
from google.colab import drive, files
from tensorflow import keras
import numpy as np
from imutils import paths
from tensorflow.keras import layers
from sklearn.metrics import plot_confusion_matrix, classification_report,confusion_matrix
from tensorflowjs.converters import save_keras_model
from tensorflow.keras.utils import to_categorical
import pandas as pd, numpy as np, os, cv2
import matplotlib.pyplot as plt
import tensorflow as tf, shutil
import splitfolders
import glob

Mount Google Drive

In [ ]:
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

logdir     = "./log"
dirRoot    = 'drive/My Drive/AKSARA-KOMERING'
dataset    = os.path.join(dirRoot, 'aksara_test')
result_no_augmented = os.path.join(dirRoot, 'result_no_augmented2')


Mounted at /content/drive


Define Some static variable

In [ ]:
batch_size = 36
epochs     = 100
IMG_HEIGHT = 48
IMG_WIDTH  = 48
n_class = 336

Build Method used to load save and convert model

Normalize data image from corrupted and bad image data

In [ ]:
from pathlib import Path
from PIL import Image

path = glob.glob(os.path.join(dataset, '*/*.*'))
# path should be list of the image files
def normalize_image_data(path):
  bad_image = []
  for index, img_p in enumerate(path):
      try:
        img = Image.open(img_p)
        img.verify()
      except (IOError, SyntaxError) as e:
        bad_image.append(img_p)
        print("bad image =>" + img_p)
  return bad_image

# bad_image = normalize_image_data(path)

In [ ]:
import json

def convert_model(model, model_name):

  path_name = os.path.join(result_no_augmented, model_name)
  if not os.path.exists(path_name): os.makedirs(path_name)

  save_keras_model(model, path_name)

def save_model_label(data, model_name):
  path_name = os.path.join(result_no_augmented, model_name)

  if not os.path.exists(path_name): os.makedirs(path_name)

  name = os.path.join(path_name, 'label.json')

  with open(name, 'w') as f :
    json.dump(data, f)


# call this method to save model
# it will save label and convert model

def save_model(model,data_label, model_name):

  path_name = os.path.join(result_no_augmented, model_name)
  if not os.path.exists(path_name): os.makedirs(path_name)

  model.save(os.path.join(path_name, 'model.h5'))

  # convert model
  convert_model(model, model_name)

  save_model_label(data_label, model_name)

  # save model wich other configuration
  tf.keras.models.save_model(
      model,
      os.path.join(path_name, 'pb_model'),
      overwrite=True,
      include_optimizer=True,
      save_format=None,
      signatures=None,
      options=None
  )

def load_model(model_name) :
  path_name = os.path.join(result_no_augmented, '{}/assets'.format(model_name))
  model_name = os.path.join(path_name, 'model.h5')

  # throw an exception if model is not found in this path
  # print(model_name)
  # if not os.path.exists(model_name) : raise Exception('model not found')

  return tf.keras.models.load_model(model_name)

In [ ]:
import shutil

sample_aksara = {}

for item in path:
  label = item.split('/')[-2]
  if label not in sample_aksara:
    sample_aksara[label] = item
  # else:
    # print("aksara => {} sudah ada di diretory".format(sample_aksara[label]))
# creaet sample folder which is containt one image aksara per label
# store it into sample folder

sample_path = os.path.join(dirRoot, 'sample')
if not os.path.exists(sample_path): os.makedirs(sample_path)

# len(sample_aksara.keys())
# copy image into sample directory

# for item in sample_aksara:
#   label = item.replace(' ', '-')
#   dest = os.path.join(sample_path, "{}.png".format(label))

#   image_source = cv2.imread(sample_aksara[item])
#   image_source = cv2.resize(image_source, (IMG_HEIGHT, IMG_WIDTH))
#   image_source = cv2.cvtColor(image_source, cv2.COLOR_BGR2GRAY)
#   cv2.imwrite(dest, image_source)
  # cv2_imshow(image_source)
  # break
data = {}

for item in sample_aksara:
  label = item.replace(' ', '-')
  data[item] = "{}.png".format(label)
json.dumps(data)

'{"a": "a.png", "a\'": "a\'.png", "ah": "ah.png", "ai": "ai.png", "an": "an.png", "ang": "ang.png", "ar": "ar.png", "au": "au.png", "b": "b.png", "ba": "ba.png", "bah": "bah.png", "bai": "bai.png", "ban": "ban.png", "bang": "bang.png", "bar": "bar.png", "bau": "bau.png", "be": "be.png", "bi": "bi.png", "bo": "bo.png", "bu": "bu.png", "c": "c.png", "cah": "cah.png", "cai": "cai.png", "can": "can.png", "cang": "cang.png", "car": "car.png", "ce": "ce.png", "ci": "ci.png", "co": "co.png", "cu": "cu.png", "d": "d.png", "da": "da.png", "dah": "dah.png", "dai": "dai.png", "dan": "dan.png", "dang": "dang.png", "dar": "dar.png", "dau": "dau.png", "de": "de.png", "di": "di.png", "do": "do.png", "du": "du.png", "e": "e.png", "g": "g.png", "ga": "ga.png", "gah": "gah.png", "gai": "gai.png", "gan": "gan.png", "gang": "gang.png", "gar": "gar.png", "gau": "gau.png", "ge": "ge.png", "gi": "gi.png", "go": "go.png", "gu": "gu.png", "h": "h.png", "ha": "ha.png", "hah": "hah.png", "hai": "hai.png", "han":

## Build a method used to read dataset with augmented or not augmented **bold text**

In [ ]:
# read dataset
def read_dataset(batch_size=36):

  # check if read an image with augmented generator or not
  test = tf.keras.preprocessing.image_dataset_from_directory(
      dataset,
      labels="inferred",
      label_mode="int",
      class_names=None,
      color_mode="grayscale",
      batch_size= batch_size,
      image_size=(IMG_HEIGHT, IMG_WIDTH),
      shuffle=False,
      seed=None
  )

  # configure the dataset for performance
  AUTOTUNE = tf.data.AUTOTUNE

 # test_ds = val.cache().prefetch(buffer_size=AUTOTUNE)

  return test

In [ ]:
test = read_dataset()

Found 17911 files belonging to 336 classes.


In [ ]:
class_names = test.class_names
print(class_names)


['a', "a'", 'ah', 'ai', 'an', 'ang', 'ar', 'au', 'b', 'ba', 'bah', 'bai', 'ban', 'bang', 'bar', 'bau', 'be', 'bi', 'bo', 'bu', 'c', 'ca', 'cah', 'cai', 'can', 'cang', 'car', 'cau', 'ce', 'ci', 'co', 'cu', 'd', 'da', 'dah', 'dai', 'dan', 'dang', 'dar', 'dau', 'de', 'di', 'do', 'du', 'e', 'g', 'ga', 'gah', 'gai', 'gan', 'gang', 'gar', 'gau', 'ge', 'gi', 'go', 'gu', 'h', 'ha', 'hah', 'hai', 'han', 'hang', 'har', 'hau', 'he', 'hi', 'ho', 'hu', 'i', 'j', 'ja', 'jah', 'jai', 'jan', 'jang', 'jar', 'jau', 'je', 'ji', 'jo', 'ju', 'k', 'ka', 'kah', 'kai', 'kan', 'kang', 'kar', 'kau', 'ke', 'ki', 'ko', 'ku', 'l', 'la', 'lah', 'lai', 'lan', 'lang', 'lar', 'lau', 'le', 'li', 'lo', 'lu', 'm', 'ma', 'mah', 'mai', 'man', 'mang', 'mar', 'mau', 'mb', 'mba', 'mbah', 'mbai', 'mban', 'mbang', 'mbar', 'mbau', 'mbe', 'mbi', 'mbo', 'mbu', 'me', 'mi', 'mo', 'mp', 'mpa', 'mpah', 'mpai', 'mpan', 'mpang', 'mpar', 'mpau', 'mpe', 'mpi', 'mpo', 'mpu', 'mu', 'n', 'na', 'nah', 'nai', 'nan', 'nang', 'nar', 'nau', 'nc',

In [ ]:

file_name = "model-no-agumented-komering-acurracy-train0.9900427460670471-accuracy-val0.9658023118972778"
model = load_model(file_name)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 48)        480       
                                                                 
 batch_normalization (BatchN  (None, 48, 48, 48)       192       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 48, 48, 48)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 24, 48)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 24, 24, 48)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 128)       1

### Build an method which is used to train a model

In [ ]:
def normalize_model_label(data):
  result = {}
  for i, item in enumerate(data.class_names):
    result[item] = i
  return result

label = normalize_model_label(test)

In [ ]:
print(label)

{'a': 0, "a'": 1, 'ah': 2, 'ai': 3, 'an': 4, 'ang': 5, 'ar': 6, 'au': 7, 'b': 8, 'ba': 9, 'bah': 10, 'bai': 11, 'ban': 12, 'bang': 13, 'bar': 14, 'bau': 15, 'be': 16, 'bi': 17, 'bo': 18, 'bu': 19, 'c': 20, 'ca': 21, 'cah': 22, 'cai': 23, 'can': 24, 'cang': 25, 'car': 26, 'cau': 27, 'ce': 28, 'ci': 29, 'co': 30, 'cu': 31, 'd': 32, 'da': 33, 'dah': 34, 'dai': 35, 'dan': 36, 'dang': 37, 'dar': 38, 'dau': 39, 'de': 40, 'di': 41, 'do': 42, 'du': 43, 'e': 44, 'g': 45, 'ga': 46, 'gah': 47, 'gai': 48, 'gan': 49, 'gang': 50, 'gar': 51, 'gau': 52, 'ge': 53, 'gi': 54, 'go': 55, 'gu': 56, 'h': 57, 'ha': 58, 'hah': 59, 'hai': 60, 'han': 61, 'hang': 62, 'har': 63, 'hau': 64, 'he': 65, 'hi': 66, 'ho': 67, 'hu': 68, 'i': 69, 'j': 70, 'ja': 71, 'jah': 72, 'jai': 73, 'jan': 74, 'jang': 75, 'jar': 76, 'jau': 77, 'je': 78, 'ji': 79, 'jo': 80, 'ju': 81, 'k': 82, 'ka': 83, 'kah': 84, 'kai': 85, 'kan': 86, 'kang': 87, 'kar': 88, 'kau': 89, 'ke': 90, 'ki': 91, 'ko': 92, 'ku': 93, 'l': 94, 'la': 95, 'lah': 96,

In [ ]:
test_labels = np.array([])
for x, y in test:
   test_labels = np.concatenate([test_labels, y.numpy()])
test_labels.shape # sanity check better match test_predicted_labels


(17911,)

In [ ]:
print(test_labels)

[  0.   0.   0. ... 335. 335. 335.]


In [ ]:
test_acc = model.evaluate(test)
dict(zip(model.metrics_names, test_acc))

498/498 [==============================] - 18s 16ms/step - loss: 1.5689 - accuracy: 0.9243


{'loss': 1.5688695907592773, 'accuracy': 0.9243481755256653}

In [ ]:
pred_label=model.predict(test)
preds = model.predict(test)
pred_lbls = np.argmax(preds, axis=1)
#true_lbls = np.argmax(ytrain, axis=1)



In [ ]:
print(pred_lbls)

[  0   0   0 ... 335 335 335]


In [ ]:
#testY=test_df['label'].map(reverse_mapping)
from sklearn.metrics import classification_report
print(classification_report(test_labels,pred_lbls,target_names=class_names,digits=5))

              precision    recall  f1-score   support

           a    0.85106   1.00000   0.91954        40
          a'    0.97368   0.92500   0.94872        40
          ah    1.00000   0.95000   0.97436        40
          ai    0.96875   0.77500   0.86111        40
          an    0.87879   0.72500   0.79452        40
         ang    0.76000   0.95000   0.84444        40
          ar    1.00000   0.92500   0.96104       120
          au    1.00000   0.95000   0.97436        40
           b    0.97500   0.97500   0.97500        40
          ba    0.86957   1.00000   0.93023        40
         bah    1.00000   1.00000   1.00000        40
         bai    0.97561   1.00000   0.98765        40
         ban    0.91892   0.85000   0.88312        40
        bang    0.76471   0.97500   0.85714        40
         bar    0.97436   0.95000   0.96203       120
         bau    1.00000   0.92500   0.96104        40
          be    0.93421   0.88750   0.91026        80
          bi    0.98649   0